# Small Molecular System Simulation with OQD Analog Interface

This tutorial demonstrates how to simulate a minimal model of a small molecule (e.g., H₂) using the OQD analog interface and oqd-analog-emulator. We construct a two-qubit Hamiltonian representing the electronic structure in a minimal basis, simulate its time evolution, and analyze observables such as population and entanglement entropy.

References:
- See e.g. McArdle et al., Rev. Mod. Phys. 92, 015003 (2020) for quantum simulation of chemistry.

## 1. Introduction

Quantum simulation of molecules is a key application of quantum computing. Here, we use a minimal two-qubit model to represent the H₂ molecule in a minimal basis. The Hamiltonian is expressed in terms of Pauli operators, and we simulate its dynamics using the OQD analog interface.

## 2. Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from oqd_core.interface.analog.operator import PauliI, PauliX, PauliZ
from oqd_core.interface.analog.operation import AnalogCircuit, AnalogGate
from oqd_core.backend.metric import Expectation, EntanglementEntropyVN
from oqd_core.backend.task import Task, TaskArgsAnalog
from oqd_analog_emulator.qutip_backend import QutipBackend

## 3. Define the Hamiltonian

We use a simple two-qubit Hamiltonian as a minimal model for H₂:

$$
H = c_0 I + c_1 Z_0 + c_2 Z_1 + c_3 Z_0 Z_1 + c_4 X_0 X_1
$$

where $I$ is the identity, $Z$ and $X$ are Pauli operators, and the coefficients are chosen for illustration.

In [ ]:
from functools import reduce

def tensor(ops):
    return reduce(lambda a, b: a @ b, ops)

# Example coefficients for minimal H2
c0 = -1.0
c1 = 0.5
c2 = 0.5
c3 = 0.3
c4 = 0.2

n = 2  # two qubits for minimal H₂

H = (
    c0 * tensor([PauliI(), PauliI()]) +
    c1 * tensor([PauliZ(), PauliI()]) +
    c2 * tensor([PauliI(), PauliZ()]) +
    c3 * tensor([PauliZ(), PauliZ()]) +
    c4 * tensor([PauliX(), PauliX()])
)

## 4. Prepare the Initial State

We prepare the system in the $|01\rangle$ state (electron in orbital 1).

In [ ]:
init_gate = tensor([PauliI(), PauliX()])

## 5. Set Up the Analog Quantum Circuit

In [ ]:
circuit = AnalogCircuit()
circuit.evolve(duration=np.pi/2, gate=AnalogGate(hamiltonian=init_gate))  # prepare |01>
circuit.evolve(duration=8, gate=AnalogGate(hamiltonian=H))
circuit.measure()

## 6. Define Observables and Metrics

We measure population observables and entanglement entropy.

In [ ]:
args = TaskArgsAnalog(
    n_shots=1000,
    fock_cutoff=1,
    metrics={
        "Z0": Expectation(operator=tensor([PauliZ(), PauliI()])),
        "Z1": Expectation(operator=tensor([PauliI(), PauliZ()])),
        "ZZ": Expectation(operator=tensor([PauliZ(), PauliZ()])),
        "XX": Expectation(operator=tensor([PauliX(), PauliX()])),
        "S": EntanglementEntropyVN(qreg=[0])
    },
    dt=1e-2,
)

## 7. Run the Simulation

In [ ]:
task = Task(program=circuit, args=args)
backend = QutipBackend()
results = backend.run(task=task)

## 8. Plot Results

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(8, 5))
plt.plot(results.times, results.metrics["Z0"], label=r"$\langle Z_0 \rangle$")
plt.plot(results.times, results.metrics["Z1"], label=r"$\langle Z_1 \rangle$")
plt.plot(results.times, results.metrics["ZZ"], label=r"$\langle Z_0 Z_1 \rangle$")
plt.plot(results.times, results.metrics["XX"], label=r"$\langle X_0 X_1 \rangle$")
plt.plot(results.times, results.metrics["S"], label="Entanglement $S$")
plt.xlabel("Time")
plt.ylabel("Observable")
plt.legend()
plt.title("Dynamics of a Minimal H₂ Molecular Model")
plt.tight_layout()
plt.show()

## 9. Discussion

This simulation demonstrates the time evolution of a minimal molecular system (H₂) using the OQD analog interface. You can see the dynamics of local observables and entanglement entropy, which are important for understanding quantum chemistry on quantum devices.

For more details on quantum simulation of chemistry, see McArdle et al., Rev. Mod. Phys. 92, 015003 (2020).